In [1]:
import pickle
import os
os.chdir('/mnt/data/wangw/AFS/赵路晴/test')
filepath = '/mnt/data/wangw/AFS/赵路晴/test/test_adata_combined.h5ad'
file ='/mnt/data/wangw/AFS/赵路晴/test/model_reult.pickle'
model_reult = pickle.load(open(file,'rb'))
import scanpy as sc
count = 0
adata = None
if os.path.exists("test_adata_combined.pickle"):
    adata = pickle.load(open("test_adata_combined.pickle",'rb'))
else:
    adata = sc.read_h5ad(filepath)
    with open("test_adata_combined.pickle", "wb") as f:
        pickle.dump(adata, f)
while not adata:
    try:
        adata = sc.read_h5ad(filepath)
        with open("test_adata_combined.pickle", "wb") as f:
            pickle.dump(adata, f)
    except Exception as e:
        count+=1
        print('try num :',count)

acc, auc_val, fpr, tpr, model, X_test, adata_test, le,metrics_list,types_clean,dflist =model_reult['all']['XGBoost']
print(adata)

AnnData object with n_obs × n_vars = 286843 × 28913
    obs: 'sample', 'tissue', 'patient', 'n_genes_by_counts', 'total_counts', 'total_counts_rp', 'pct_counts_rp', 'total_counts_mt', 'pct_counts_mt', 'total_counts_hb', 'pct_counts_hb', 'total_counts_hsp', 'pct_counts_hsp', 'doublet_scores', 'predicted_doublets', 'doublet_info', 'n_genes', 'S_score', 'G2M_score', 'phase', 'UMAP_1', 'UMAP_2', 'sub_cluster', 'major_cluster', 'batch', 'cell_type', 'response', 'pre_post'


In [2]:
len(adata_test.var),len(adata.var)

(1000, 28913)

In [3]:
adata.obs['Cell_Type'] = adata.obs['sub_cluster'].apply(lambda x:x.split('_')[0])
# 检查 pre_post 列是否存在
import sys
import pandas as pd
if 'pre_post' not in adata.obs.columns:
    print("adata.obs 中缺少 'pre_post' 列，程序退出")
    sys.exit()
print(f"pre_post 列的唯一值: {adata.obs['pre_post'].unique().tolist()}")

# 筛选 pre_post == 'Pre' 的行
adata_ = adata[adata.obs['pre_post'] == 'pre'].copy()
print(f"筛选后 pre_post == 'pre' 的样本量: {adata_.shape[0]}")
adata_ = adata_[adata_.obs['pre_post'] == 'pre'].copy()
print(f"筛选后 pre_post == 'Pre' 的样本量: {adata_.shape[0]}")
pd.set_option('display.max_columns', None)
adata_.obs['Cancer_type_update'] = ['HCC']*adata_.obs.shape[0]
adata_.obs

pre_post 列的唯一值: ['post', 'pre']
筛选后 pre_post == 'pre' 的样本量: 98270
筛选后 pre_post == 'Pre' 的样本量: 98270


,sample,tissue,patient,n_genes_by_counts,total_counts,total_counts_rp,pct_counts_rp,total_counts_mt,pct_counts_mt,total_counts_hb,pct_counts_hb,total_counts_hsp,pct_counts_hsp,doublet_scores,predicted_doublets,doublet_info,n_genes,S_score,G2M_score,phase,UMAP_1,UMAP_2,sub_cluster,major_cluster,batch,cell_type,response,pre_post,Cell_Type,Cancer_type_update
P16-pre-P-CD8p2-GGCGACTTCCACGTTC-1,P16-pre-P-CD8p,P,P16,1427,4671.0,1542.0,33.012203,154.0,3.296938,0.0,0.0,13.0,0.278313,0.087618,False,False,1427,0.037808,-0.000656,S,-0.212849,2.458946,CD8_C03_CX3CR1,NaN,cd8,CD8T,non-responder,pre,CD8,HCC
P16-pre-P-CD8p1-CACCACTCAACAACCT-1,P16-pre-P-CD8p,P,P16,1835,7281.0,3163.0,43.441833,264.0,3.625876,0.0,0.0,15.0,0.206016,0.112999,False,False,1835,-0.002969,-0.124453,G1,7.944293,-0.144074,CD8_C01_LEF1,NaN,cd8,CD8T,non-responder,pre,CD8,HCC
P16-pre-P-CD8n-TATTACCGTCCAACTA-1,P16-pre-P-CD8n,P,P16,959,2529.0,996.0,39.383156,231.0,9.134046,0.0,0.0,8.0,0.316331,0.133690,False,False,959,-0.188267,-0.082084,G1,5.682766,3.407964,CD8_C02_GPR183,NaN,cd8,CD8T,non-responder,pre,CD8,HCC
P16-pre-T-CD8p-CTAATGGGTTCCATGA-1,P16-pre-T-CD8p,T,P16,1081,2505.0,496.0,19.800400,72.0,2.874251,0.0,0.0,69.0,2.754491,0.064125,False,False,1081,0.139061,0.017359,S,0.651383,-3.216115,CD8_C08_GZMK,NaN,cd8,CD8T,non-responder,pre,CD8,HCC
P16-pre-P-CD8p1-TGACTTTAGCGTGAGT-1,P16-pre-P-CD8p,P,P16,894,1815.0,353.0,19.449036,205.0,11.294765,0.0,0.0,9.0,0.495868,0.222841,False,False,894,-0.032617,-0.149058,G1,0.857628,-1.018432,CD8_C08_GZMK,NaN,cd8,CD8T,non-responder,pre,CD8,HCC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P18-pre-P-CTCTGGTCAGCGTAAG-1,P18-pre-P,P,P18,1396,2318.0,214.0,9.232097,96.0,4.141501,0.0,0.0,26.0,1.121657,0.137306,False,False,1396,0.041544,-0.059760,S,7.865705,-0.785990,CD8_C03_CX3CR1,CD8T,cd45,CD45,responder,pre,CD8,HCC
P27-pre-T-TGAGGGAAGAAGGCCT-1,P27-pre-T,T,P27,1319,3377.0,1045.0,30.944624,137.0,4.056855,0.0,0.0,131.0,3.879183,0.045350,False,False,1319,0.100014,0.030680,S,3.888998,1.845784,CD4_C03_CD44,CD4T,cd45,CD45,responder,pre,CD4,HCC
P27-pre-T-CGTAGGCTCGCATGGC-1,P27-pre-T,T,P27,1205,2150.0,80.0,3.720930,82.0,3.813953,0.0,0.0,181.0,8.418605,0.066667,False,False,1205,-0.219012,-0.135053,G1,9.128882,4.708649,NK_C03_GZMK,ILC,cd45,CD45,responder,pre,NK,HCC
P11-pre-P-ACACCCTCAAGGCTCC-1,P11-pre-P,P,P11,2415,6466.0,1061.0,16.408909,193.0,2.984844,0.0,0.0,49.0,0.757810,0.120760,False,False,2415,-0.131148,1.040371,G2M,4.558650,1.222805,NK_C01_FCGR3A,ILC,cd45,CD45,responder,pre,NK,HCC


In [4]:
adata_test.obs.index


Index(['P15-pre-P-GACGGCTGTACAGACG-1', 'P11-pre-P-GAGGTGATCTCCGGTT-1',
       'P18-pre-T-CTGAAACTCCAGTATG-1', 'P5-pre-P-ATGGGAGTCTGGTTCC-1',
       'P1-pre-P-CGAATGTCAGACAGGT-1', 'P16-pre-P-CD8p2-CGACCTTGTGCGGTAA-1',
       'P18-pre-T-TGAGCATGTCGAATCT-1', 'P3-pre-P-CD8p-GTCCTCATCATACGGT-1',
       'P3-pre-P-CD8p-GTTACAGGTCCGACGT-1', 'P27-pre-T-CGAATGTGTGGACGAT-1',
       ...
       'P5-pre-T-AGATCTGCAGCTGCTG-1', 'P18-pre-T-GACGGCTAGGAATGGA-1',
       'P18-pre-T-GGGAGATAGACCCACC-1', 'P26-pre-P-CACCAGGGTTTACTCT-1',
       'P5-pre-P-TAAACCGGTATAAACG-1', 'P16-pre-P-CD8p2-TGGGAAGTCTGCGTAA-1',
       'P1-pre-P-ATGTGTGAGGTGACCA-1', 'P16-pre-P-CD8p1-GCGCAACTCAACCAAC-1',
       'P11-pre-T-ACTTGTTAGATGTGGC-1', 'P16-pre-T-CD8p-TGGCTGGCAACACCTA-1'],
      dtype='object', length=19486)

In [5]:
adata_test.obs.loc['P15-pre-P-GACGGCTGTACAGACG-1']

sample                    P15-pre-P
tissue                            P
patient                         P15
n_genes_by_counts              1622
total_counts                 4009.0
total_counts_rp               673.0
pct_counts_rp              16.78723
total_counts_mt                63.0
pct_counts_mt              1.571464
total_counts_hb                 0.0
pct_counts_hb                   0.0
total_counts_hsp               51.0
pct_counts_hsp             1.272138
doublet_scores              0.11349
predicted_doublets            False
doublet_info                  False
n_genes                        1622
S_score                   -0.186944
G2M_score                  0.105986
phase                           G2M
UMAP_1                     5.417911
UMAP_2                    -1.114475
sub_cluster           NK_C01_FCGR3A
major_cluster                   ILC
batch                          cd45
cell_type                      CD45
response              non-responder
pre_post                    

In [6]:
def get_train(adata,adata_test,cell_type):
    gene = adata_test.var_names  
    if cell_type == 'all':
        adata_ = adata[adata.obs['Cell_Type'].isin(model_reult.keys()),gene]
    else:
        adata_ = adata[adata.obs['Cell_Type'] == cell_type,gene]
    test_id = adata_test.obs.index
    train_id = set(adata_.obs.index) - set(test_id)
    adata_train =  adata_[list(train_id), :]
    # print(adata_.shape,len(test_id),len(train_id))
    return adata_train
acc, auc_val, fpr, tpr, model, X_test, adata_test, le,metrics_list,types_clean,dflist =model_reult['all']['XGBoost']
get_train(adata_,adata_test,'all') 

View of AnnData object with n_obs × n_vars = 77941 × 1000
    obs: 'sample', 'tissue', 'patient', 'n_genes_by_counts', 'total_counts', 'total_counts_rp', 'pct_counts_rp', 'total_counts_mt', 'pct_counts_mt', 'total_counts_hb', 'pct_counts_hb', 'total_counts_hsp', 'pct_counts_hsp', 'doublet_scores', 'predicted_doublets', 'doublet_info', 'n_genes', 'S_score', 'G2M_score', 'phase', 'UMAP_1', 'UMAP_2', 'sub_cluster', 'major_cluster', 'batch', 'cell_type', 'response', 'pre_post', 'Cell_Type', 'Cancer_type_update'

In [7]:
adata[adata.obs['Cell_Type'].isin(model_reult.keys())]

View of AnnData object with n_obs × n_vars = 285211 × 28913
    obs: 'sample', 'tissue', 'patient', 'n_genes_by_counts', 'total_counts', 'total_counts_rp', 'pct_counts_rp', 'total_counts_mt', 'pct_counts_mt', 'total_counts_hb', 'pct_counts_hb', 'total_counts_hsp', 'pct_counts_hsp', 'doublet_scores', 'predicted_doublets', 'doublet_info', 'n_genes', 'S_score', 'G2M_score', 'phase', 'UMAP_1', 'UMAP_2', 'sub_cluster', 'major_cluster', 'batch', 'cell_type', 'response', 'pre_post', 'Cell_Type'

In [8]:
train_data = {}
for types in model_reult:
    acc, auc_val, fpr, tpr, model, X_test, adata_test, le,metrics_list,types_clean,dflist =model_reult[types]['XGBoost']
    adata_train = get_train(adata_,adata_test,'all')
    X_train = adata_train.X.toarray()   
    y_train = [1 if i == 'responder' else 0 for i in adata_train.obs['response'].values]
    train_data[types] = {"adata_train":adata_train,
                        'X_train':X_train,
                        'y_train':y_train}


In [9]:
import pickle
def read_pickle(file:str):
    data = pickle.load(open(file,'rb'))
    return data
def to_pickle(data,file):
    pickle.dump(data,open(file,'wb'))
# to_pickle(train_data,'/mnt/data/wangw/AFS/赵路晴/new_data/train_shap/train_data.pickle')

In [10]:
train_data_clean = {}
for types in model_reult:
    acc, auc_val, fpr, tpr, model, X_test, adata_test, le,metrics_list,types_clean,dflist =model_reult[types]['XGBoost']
    adata_train = get_train(adata_,adata_test,'all')
    X_train = adata_train.X.toarray()   
    y_train = [1 if i == 'responder' else 0 for i in adata_train.obs['response'].values]
    train_data_clean[types] = {'gene':adata_test.var_names.tolist(),
                        'X_train':X_train,
                        'y_train':y_train}
# to_pickle(train_data_clean,'/mnt/data/wangw/AFS/赵路晴/new_data/train_shap/train_data_clean.pickle')

In [11]:
type(adata_test.var.index.tolist()), type(adata_test.var_names)

(list, pandas.core.indexes.base.Index)

In [12]:
import pickle
def read_pickle(file:str):
    data = pickle.load(open(file,'rb'))
    return data
def to_pickle(data,file):
    pickle.dump(data,open(file,'wb'))
file ='/mnt/data/wangw/AFS/赵路晴/test/model_reult.pickle'
model_reult = pickle.load(open(file,'rb'))
train_data_clean = read_pickle('/mnt/data/wangw/AFS/赵路晴/new_data/train_shap/train_data_clean.pickle')
import os
import pandas as pd
ENSEMBL_MAP = pd.read_csv("/mnt/data/wangw/ensemble-tran.csv")
ENSEMBL_DICT = dict(zip(ENSEMBL_MAP['ensembl'], ENSEMBL_MAP['feature']))
os.environ["SCIPY_ARRAY_API"] = "1"
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, train_test_split
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score,roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import shap
import pickle
def shap_analysis(model, X_test, gene, types,model_name='XGBoots'):
    file =f'/mnt/data/wangw/AFS/赵路晴/new_data/train_shap/feature_importance_{model_name}_{types}.pickle'
    if not os.path.exists(file):
        feature_names = gene
        if isinstance(model, (RandomForestClassifier, LGBMClassifier, XGBClassifier, CatBoostClassifier)):
            explainer = shap.TreeExplainer(model, feature_names=feature_names)
        elif isinstance(model, LogisticRegression):
            # explainer = shap.LinearExplainer(model, X_test, feature_perturbation="interventional")
            explainer = shap.LinearExplainer(model, X_test)
        else:
            print(f"[警告] {str(model).split('(')[0]} 模型类型暂不支持 SHAP")
            # return None
        # explainer = shap.TreeExplainer(model, model_output='probability')
        # shap_values = explainer(X_test, check_additivity=False)
        shap_values = explainer(X_test)
        shap_vals = shap_values.values
        if shap_vals.shape[1] == X_test.shape[1] and shap_vals.ndim == 3:  # 特征在中间（错误）
            shap_vals = np.transpose(shap_vals, (0, 2, 1))
        # if shap_vals.ndim == 3:
            shap_mean = np.abs(shap_vals).mean(axis=(0, 1))
        else:
            shap_mean = np.abs(shap_vals).mean(axis=0)
        
        # print("shap_values.values shape:", shap_values.values.shape)
        # mean over samples & classes → shape (1000,)
        ensembl_ids = gene
        gene_names = [ENSEMBL_DICT.get(eid, eid) for eid in ensembl_ids]
        feature_importance = pd.DataFrame({
            'Feature': gene_names,
            'Mean SHAP Value': shap_mean
        }).sort_values(by='Mean SHAP Value', ascending=False).reset_index(drop=True)
        with open(file,'wb')as f:
            pickle.dump(feature_importance,f)
    else:
        feature_importance = pickle.load(open(file,'rb'))
    if not os.path.exists(f'/mnt/data/wangw/AFS/赵路晴/new_data/train_shap/{model_name}_shap_{types}.pdf'):
        plt.figure()
        shap.summary_plot(shap_values, X_test, feature_names=gene_names, plot_type='bar', show=False)
        plt.title(f'SHAP Summary - {types}')
        plt.tight_layout()
        plt.savefig(f'/mnt/data/wangw/AFS/赵路晴/new_data/train_shap/{model_name}_shap_{types}.pdf')
        plt.close()
    if not os.path.exists(f'/mnt/data/wangw/AFS/赵路晴/new_data/train_shap/{model_name}_shap_{types}.dot.pdf'):
        print(f'{model_name}_shap_{types}.dot.pdf')
        shap.summary_plot(shap_values, X_test, plot_type="dot", show=False)
        # fig = plt.gcf()
        plt.savefig(f'/mnt/data/wangw/AFS/赵路晴/new_data/train_shap/{model_name}_shap_{types}.dot.pdf')
        plt.close()
    return feature_importance.head(10)


# for types in model_reult:
#     for model_name in model_reult[types]:
#         print(types,model_name)
#         acc, auc_val, fpr, tpr, model, X_test, adata_test, le,metrics_list,types_clean,dflist =model_reult[types][model_name]
#         adata_train = train_data[types]['adata_train']
#         X_train = train_data[types]['X_train']
#         y_train = train_data[types]['y_train']
#         shap_analysis(model, X_train, adata_train, types,model_name=model_name)
        # shap_analysis(model, X_test, adata_test, types,model_name=model_name)

In [13]:
from multiprocessing import Pool
from joblib import Parallel, delayed
from tqdm.contrib.concurrent import process_map
def worker(args):
    # 拆包参数
    model, X_train, gene, types, model_name = args
    print(model_name,types)
    try:
        shap_analysis(model, X_train, gene, types, model_name=model_name)
    except Exception as e:
        print(e)

# 构建参数列表
task_list = []

for types in model_reult:
    for model_name in model_reult[types]:
        file =f'/mnt/data/wangw/AFS/赵路晴/new_data/train_shap/feature_importance_{model_name}_{types}.pickle'
        if not os.path.exists(file):
            acc, auc_val, fpr, tpr, model, X_test, adata_test, le, metrics_list, types_clean, dflist = model_reult[types][model_name]
            gene = train_data_clean[types]['gene']
            X_train = train_data_clean[types]['X_train']
            y_train = train_data_clean[types]['y_train']
            task_list.append((model, X_train, gene, types, model_name))
len(task_list)
# print('start run!')
# pool = Pool(16)
# # # # 多进程执行
# # with Pool(processes=4) as pool:  # 设置进程数，例如 4
# pool.map(worker, task_list)
# process_map(worker, task_list,max_workers=4)

0

[0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,


In [ ]:
path  = '/mnt/data/wangw/AFS/赵路晴/new_data/train_shap'
os.chdir(path)
import glob
# for file in os.listdir(path):
# for file in glob.glob(f'feature*.pickle'):
#     feature_df = read_pickle(file)
#     feature_df.to_csv(file.replace('pickle','csv'),index=False)
    

In [14]:
path  = '/mnt/data/wangw/AFS/赵路晴/new_data/train_shap'
os.chdir(path)
import glob

In [24]:
N = 5
top_features = feature_df['Feature'][:N].values
top_features

array(['CD69', 'RGS1', 'FTH1', 'CCL4L2', 'B2M'], dtype=object)

In [36]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    roc_curve,confusion_matrix)
from sklearn.model_selection import train_test_split, RandomizedSearchCV
plt.rcParams['font.family'] = 'Arial'
model_param_grid = {
    "Random Forest": (RandomForestClassifier(random_state=42), {
        'n_estimators': [50, 100],
        'max_depth': [None, 5, 10]
    }),
    "Logistic Regression": (LogisticRegression(max_iter=1000), {
        'C': [0.1, 1, 10]
    }),
    "LightGBM": (LGBMClassifier(verbose=-1,random_state=42,device='gpu'), {
        'n_estimators': [50, 100],
        'max_depth': [5, 10]
    }),
    "CatBoost": (CatBoostClassifier(task_type='GPU',verbose=0, random_state=42), {
        'iterations': [100, 200],
        'depth': [4, 6]
    }),
    "XGBoost": (XGBClassifier(tree_method = "hist", device = "cuda", eval_metric='logloss', random_state=42), {
        "max_depth": [
            3,
            5,
            7,
            10
        ],
        "learning_rate": [
            0.01,
            0.05,
            0.1,
            0.3
        ],
        "n_estimators": [
            50,
            100,
            200,
            300
        ],
        "reg_lambda": [
            0.1,
            1.0,
            10.0,
            100.0
        ],
        "reg_alpha": [
            0.0,
            0.5,
            1.0,
            5.0
        ],
        "subsample": [
            0.6,
            0.8,
            1.0
        ],
        "colsample_bytree": [
            0.6,
            0.8,
            1.0
        ]
    }),
 
}
random_seed =42
# 假设 shap_importance 是 pd.Series，index 是特征名，value 是 mean(|SHAP|)
def compute_roc_auc_ci(y_true, y_scores, n_bootstraps=1000, alpha=0.95):
    rng = np.random.RandomState(random_seed)
    bootstrapped_scores = []
    # y_true = y_true.to_numpy() if isinstance(y_true, pd.Series) else y_true
    y_true = np.array(y_true)
    for _ in range(n_bootstraps):
        indices = rng.randint(0, len(y_scores), len(y_scores))
        if len(np.unique(y_true[indices])) < 2:
            continue
        score = roc_auc_score(y_true[indices], y_scores[indices])
        bootstrapped_scores.append(score)

    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()
    lower = sorted_scores[int((1.0 - alpha) / 2 * len(sorted_scores))]
    upper = sorted_scores[int((1.0 + alpha) / 2 * len(sorted_scores))]
    return lower, upper

def plot_merge(feature_df,cell,X_train,y_train,X_test,y_test):
    results = []
    top_feature_counts = [5, 10, 20, 30, 50, 100,200]
    plt.figure(figsize=(10,8))
    for N in top_feature_counts:
        
        top_features = feature_df['Feature'][:N].values
        X_train_ = X_train[top_features]
        X_test_ = X_test[top_features]
        # clf = XGBClassifier( eval_metric='logloss', random_state=42)
        
        cv = 5
        model,param_grid = model_param_grid[model_name]
        grid = RandomizedSearchCV(model, param_grid, cv=cv, scoring='roc_auc', n_jobs=1, verbose=0) #gpu
        grid.fit(X_train_, y_train)
        print(f"Best AUROC (CV): {grid.best_score_:.4f}")
        print(f"Best Params: {grid.best_params_}")
        best_model = grid.best_estimator_
        y_test_pred = best_model.predict(X_test_)
        y_test_proba = best_model.predict_proba(X_test_)[:, 1]
        test_acc = accuracy_score(y_test, y_test_pred)
        test_fpr, test_tpr, _ = roc_curve(y_test, y_test_proba)
        test_auc = auc(test_fpr, test_tpr)
        # auc_scores = cross_val_score(clf, X_sub, y_train, cv=StratifiedKFold(5), scoring='roc_auc')
        results.append({
            "topN": N,
            "auc": test_acc,
            "roc_curve": (test_fpr, test_tpr)
        })
        # results.append({
        #     "topN": N,
        #     "mean_auc": auc_scores.mean(),
        #     "std_auc": auc_scores.std()
        # })
    # 结果可视化
        plt.plot(test_fpr, test_tpr, lw=2, label=f"{model_name} (AUC = {test_acc:.3f})")

    plt.plot([0,1], [0,1], color="gray", lw=1.5, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate (1 - Specificity)")
    plt.ylabel("True Positive Rate (Sensitivity)")
    plt.title(f"ROC Curves of top {N} SHAP Features : {cell}_{model_name}")
    plt.grid(alpha=0.3)
    plt.tight_layout()
    # plt.save_fig()
    plt.show()
    plt.close()
for types in model_reult:
    # for model_name in model_reult[types]:
    model_name = 'XGBoost'
    file =f'/mnt/data/wangw/AFS/赵路晴/new_data/train_shap/feature_importance_{model_name}_{types}.pickle'
    feature_df = read_pickle(file)

    gene = train_data_clean[types]['gene']
    acc, auc_val, fpr, tpr, model, X_test, adata_test, le, metrics_list, types_clean, dflist = model_reult[types][model_name]
    X_train = train_data_clean[types]['X_train']
    X_train = pd.DataFrame(X_train,columns=gene)
    X_test= pd.DataFrame(X_test,columns=gene)
    y_train = train_data_clean[types]['y_train']
    y_test = [1 if i == 'responder' else 0 for i in adata_test.obs['response'].values]
    # plot_merge(feature_df,types,X_train,y_train,X_test,y_test)
    result = {}
    top_feature_counts = [5, 10, 20, 50, 100,200]
    plt.figure(figsize=(10,8))
    # with open()as f:
    for N in top_feature_counts:
        
        top_features = feature_df['Feature'][:N].values
        X_train_ = X_train[top_features]
        X_test_ = X_test[top_features]
        # clf = XGBClassifier( eval_metric='logloss', random_state=42)
        
        cv = 5
        model,param_grid = model_param_grid[model_name]
        grid = RandomizedSearchCV(model, param_grid, cv=cv, scoring='roc_auc', n_jobs=1, verbose=0) #gpu
        grid.fit(X_train_, y_train)
        # print(f"Best AUROC (CV): {grid.best_score_:.4f}")
        # print(f"Best Params: {grid.best_params_}")
        best_model = grid.best_estimator_
        y_test_pred = best_model.predict(X_test_)
        y_test_proba = best_model.predict_proba(X_test_)[:, 1]
        test_acc = accuracy_score(y_test, y_test_pred)
        fpr, tpr, thresholds = roc_curve(y_test, y_test_proba)
        test_auc = auc(test_fpr, test_tpr)
        ci_lower, ci_upper = compute_roc_auc_ci(y_test, y_test_pred)
        acc = accuracy_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        recall = recall_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred)
        tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
        J_scores = tpr - fpr
        idx = np.argmax(J_scores)
        best_fpr, best_tpr, best_th = fpr[idx], tpr[idx], thresholds[idx]

        conf_matrix = {
                    "TP": int(tp),
                    "TN": int(tn),
                    "FP": int(fp),
                    "FN": int(fn)
                    }
        result['acc']=acc
        result['precision']=precision
        result['recall']=recall
        result['f1']=f1
        result['auc']=auc
        result['fpr']=fpr
        result['tpr']=tpr
        result['acc']=acc
        result['auc_ci']=(ci_lower, ci_upper)
        result['confusion_matrix']=conf_matrix
        result['best_th'] = best_th
        # auc_scores = cross_val_score(clf, X_sub, y_train, cv=StratifiedKFold(5), scoring='roc_auc')
        # results.append({
        #     "topN": N,
        #     "auc": test_acc,
        #     "roc_curve": (test_fpr, test_tpr)
        # })
    # 结果可视化
        plt.plot(fpr, tpr, lw=2, label=f"top {N}: AUC: {auc:.4f} (95% CI: {ci_lower:.4f} - {ci_upper:.4f}) | thr={best_th:.2f}")
        plt.scatter(best_fpr, best_tpr, marker="o", color="red", s=40, zorder=5)
    plt.plot([0,1], [0,1], color="gray", lw=1.5, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate (1 - Specificity)")
    plt.ylabel("True Positive Rate (Sensitivity)")
    plt.title(f"ROC Curves of top SHAP Features : {types}_{model_name}")
    plt.legend(loc="lower right", frameon=False)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    # plt.save_fig()
    plt.show()
    # plt.close()
    break


TypeError: unsupported format string passed to function.__format__

<Figure size 1000x800 with 0 Axes>

In [ ]:
len(train_data_clean[types]['gene']),

1000

In [ ]:
# Parallel(n_jobs=16)(
#     delayed(worker)(model, X_train, gene, types, model_name)
#     for types in model_reult
#     for model_name in model_reult[types]
#     for (acc, auc_val, fpr, tpr, model, X_test, adata_test, le, metrics_list, types_clean, dflist)
#         in [model_reult[types][model_name]]
#     for gene in [train_data_clean[types]['gene']]
#     for X_train in [train_data_clean[types]['X_train']]
# )

In [3]:
df = pd.read_csv('/mnt/data/wangw/AFS/赵路晴/new_data/data_with_responder/CNP0000650/Single_Cell/CSE0000008/HCC_log_tpm_expression_matrix.txt',sep='\t')
print(df.shape)
df[:5]

(19744, 16499)


,gene,P01_T_0001,P01_T_0003,P01_T_0004,P01_T_0007,P01_T_0008,P01_T_0011,P01_T_0013,P01_T_0014,P01_T_0015,...,P19_T_0124,P19_T_0126,P19_T_0128,P19_T_0129,P19_T_0134,P19_T_0138,P19_T_0139,P19_T_0143,P19_T_0144,P19_T_0146
0,OR4F5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0
1,FO538757.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0
2,FO538757.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.157,0.0,0.0,0.0,0.0,0.0,0.0677,0.0488,0.0,0.0
3,OR4F29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0
4,OR4F16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0


In [5]:
df.index= df['gene'].values
df = df.drop(columns=['gene'])
df.T[:5]

,OR4F5,FO538757.2,FO538757.1,OR4F29,OR4F16,SAMD11,NOC2L,KLHL17,PLEKHN1,PERM1,...,AC007325.4,AC007325.2,BX072566.1,AL354822.1,AC023491.2,AC004556.1,AC233755.2,AC233755.1,AC240274.1,AC213203.1
P01_T_0001,0.0,0.0,0.0,0.0,0.0,0.0,0.2070,0.0,0.0,0.0,...,2.5100,0.0,0.0,0.00,0.3920,0.0,0.0,0.0,0.0862,0.0
P01_T_0003,0.0,0.0,0.0,0.0,0.0,0.0,0.1040,0.0,0.0,0.0,...,0.0392,0.0,0.0,3.33,0.0000,0.0,0.0,0.0,0.4050,0.0
P01_T_0004,0.0,0.0,0.0,0.0,0.0,0.0,0.0770,0.0,0.0,0.0,...,0.0000,0.0,0.0,0.00,0.0392,0.0,0.0,0.0,0.0000,0.0
P01_T_0007,0.0,0.0,0.0,0.0,0.0,0.0,0.0198,0.0,0.0,0.0,...,0.0000,0.0,0.0,0.00,0.0000,0.0,0.0,0.0,0.0000,0.0
P01_T_0008,0.0,0.0,0.0,0.0,0.0,0.0,0.4640,0.0,0.0,0.0,...,0.0000,0.0,0.0,0.00,0.0000,0.0,0.0,0.0,0.0000,0.0


In [6]:
df.T.to_csv('/mnt/data/wangw/AFS/赵路晴/new_data/data_with_responder/CNP0000650/Single_Cell/CSE0000008/expression.csv')